# Introducción

En este cuaderno emplearemos la implementación de RAPIDS del algoritmo de vecinos cercanos (*K-Nearest Neighbors*) para el cálculo de distancias entre puntos de una nube bidimensional.

# Sección 1: KNN mediante GPU

## Carga y tratamiento de datos

Importaciones
* **os** y **os.path**: importan utilidades de Python para tratamiento de ficheros y comprobación de rutas.
* **cudf**, **cupy** y **cuml**: librerías de RAPIDS.
* **pandas** y **numpy**: librerías de manejo de DataFrames y gestión numérica, de vectores y tablas. Equivalentes a cudf y cupy respectivamente para CPU.
* **sklearn**: librería de manejo de modelos machine learning, equivalente de cuml en CPU.

In [267]:
import os
import os.path

import cudf
import cupy as cp
import cuml

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

Ficheros de utilidades:
* **f_northing**: conversión de coordenadas latitud-longitud a norte-este, empleados en manejo de mapas y coordenadas.
* **f_northing_numpy**: equivalente de f_northing para estructuras de numpy.
* **f_static_data**: funciones para cargar listas de datos estáticos, tales como nombres de ciudades a utilizar desde el directorio /data, columnas a leer por cada fichero CSV, y columnas a usar en el entrenamiento del modelo.
* **f_utils**: utilidades de conversión de datos a tipos float32/float64, limpiado de strings de precios, factorización de columnas complejas en mapas numéricos, etc.

In [268]:
%run ../utils/f_northing.py
%run ../utils/f_northing_numpy.py
%run ../utils/f_static_data.py
%run ../utils/f_utils.py

Inicialización de datos:
* **cities_to_use**: lista de directorios dentro de /data sobre los que leer datasets en formato CSV. Cada directorio es una ciudad, área, estado o país.
* **columns_to_use**: lista de columnas a leer dentro de cada CSV. Todas las columnas deben existir y sus nombres deben coincidir.
* **columns_to_fit**: lista de columnas a utilizar para el entrenamiento de modelos de machine learning. Nos permite hacer múltiples usos de un dataset sin necesidad de leerlo de nuevo.

In [269]:
#cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
cities_to_use = cities_to_use_1()
#cities_to_use = cities_to_use_2()

columns_to_use = ['host_id', 'host_response_rate', 'host_acceptance_rate',
                  'latitude', 'longitude', 'accommodates', 'price', 'number_of_reviews', 'reviews_per_month',
                 'neighbourhood_cleansed']

El DataFrame **listings** representa el conjunto de datos final sobre el que realizar operaciones de ciencia de datos o machine learning. Por cada directorio de datos, leemos todos los ficheros CSV disponibles y los agregamos a **listings**.
* **standard_object_type**: algunas columnas no tienen un formato único de datos; esto puede deberse a diferentes departamentos o metodologías empleadas durante los varios *scraping* de datos de la fuente. Esta función convierte columnas (detectadas manualmente a priori) en tipo genérico *object*, que después convertiremos a un tipo de datos más apropiado para nuestro uso.
* **drop_duplicates**: dado que utilizamos múltiples *datasets* por ciudades, hay listados de AirBnB que no cambian durante varios meses. En este caso, no necesitamos datos redundantes, y los removemos del DataFrame final.
* **reset_index**: los datos de cada fichero se han unido mediante un DataFrame diferente, y por lo tanto los índices del DataFrame quedan mezclados al final del proceso. Con esta instrucción forzamos el índice de **listings** a ordenarse. La instrución añade una columna adicional con el índice antiguo, que podemos obviar con el parámetro drop=**True**.
* **shape**: instrucción que devuelve el tamaño completo del DataFrame en formato (filas, columnas)

In [270]:
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings.size == 0:
                    listings = temp_df
                else:
                    for column in listings.columns:
                        if listings[column].dtype != temp_df[column].dtype:
                            print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                    listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates().reset_index(drop=True)

Tratamiento simple:
* **type_conversion**: convertimos todas las columnas a un tipo de datos común: *float32* y *float64* son los más aceptados por los algoritmos de RAPIDS.
* **column_factorize**: dado que algunos algoritmos no suportan datos no numéricos, factorizamos las columnas de texto. La operación factorize() convierte una columna con valores no numéricos a un mapa en el que cada valor único se representa por un integer. En nuestro caso, no necesitaremos el mapa que reconoce cada valor, pero es posible usarlo para devolver formato a los datos de cara a estudio o representación.

In [271]:
type_conversion(listings, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings, ['neighbourhood_cleansed'])

Tratamiento de cadenas:
* **clean_format_strings**: eliminamos el carácter '%' de campos que lo contienen, y convertimos el valor resultante a *float32*.
* **clean_format_price**: similar a la función anterior, pero eliminando caracteres de moneda (en nuestro caso, '$' y la coma de cantidades numéricas).

In [272]:
clean_format_strings(listings, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price(listings, ['price'])

A fin de visualizar los datos en un formato 2D similar a un mapa, convertimos las coordatas latitud-longitud en distancias norte-este, y les asignamos columnas nuevas.

In [273]:
cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

Visualización de las primeras 5 filas, para verificar el tratamiento de datos.

In [274]:
listings.head()

,host_id,host_response_rate,host_acceptance_rate,neighbourhood_cleansed,latitude,longitude,accommodates,price,number_of_reviews,reviews_per_month,northing,easting
0,23.0,-1.0,-1.0,31.0,30.17865,-97.74929,2.0,65.0,4.0,0.15,6796325.0,-10620581.0
1,23.0,-1.0,-1.0,31.0,30.17865,-97.74929,2.0,65.0,4.0,0.16,6796325.0,-10620581.0
2,23.0,-1.0,-1.0,31.0,30.17926,-97.74887,2.0,65.0,4.0,0.26,6796217.5,-10620283.0
3,23.0,-1.0,-1.0,31.0,30.17989,-97.74904,2.0,35.0,0.0,-1.00,6796309.0,-10620063.0
4,23.0,-1.0,-1.0,31.0,30.18025,-97.74941,2.0,40.0,0.0,-1.00,6796453.0,-10619977.0


## Aplicación de un algoritmo KNN para calcular distancias entre listados

*KNN* es un algoritmo de búsqueda de distancias entre puntos. En nuestro caso, dado que utilizaremos coordenadas reales de un mapa, estas distancias pueden usarse en contextos geográficos.

Emplearemos dos conjuntos de datos separados, uno con listados baratos (*listings_cheap*) sobre el que entrenaremos el modelo, y uno con listados más caros (*listings_expensive*) sobre el que probaremos el modelo, para recibir los listados más baratos cercanos a cada listado caro.

Parámetros:
* **n_neighbors**: especifica cuántos vecinos el modelo debe buscar sobre cada punto del *dataset*.

Funciones:
* **fit**: ejecución del algoritmo *KNN* sobre el conjunto de datos indicado.
* **kneighbors**: busca sobre el modelo entrenado los K vecinos más cercanos del conjunto de datos que se introduce.

In [275]:
%%time
knn = cuml.NearestNeighbors(n_neighbors=3)
listings_cheap = listings[listings['price'] <= 50.0].reset_index(drop=True)
listings_expensive = listings[listings['price'] >= 100.0].reset_index(drop=True)
listings_cheap_cols = listings_cheap[['easting', 'northing']]
listings_expensive_cols = listings_expensive[['easting', 'northing']]
knn.fit(listings_cheap_cols)
distances, indices = knn.kneighbors(listings_expensive_cols)

CPU times: user 12 s, sys: 3.71 s, total: 15.8 s
Wall time: 15.7 s


El conjunto **distances** devuelve las distancias de cada punto del conjunto de prueba a los K vecinos más cercanos.

In [276]:
distances

,0,1,2
0,0.000000,0.000000,0.000000e+00
1,0.000000,0.000000,0.000000e+00
2,0.000000,0.000000,0.000000e+00
3,0.000000,0.000000,0.000000e+00
4,0.000000,0.000000,0.000000e+00
...,...,...,...
694886,13584.895508,13584.895508,1.358490e+04
694887,10836.997070,16888.240234,1.688824e+04
694888,255105.515625,255105.515625,2.551384e+05
694889,55711.625000,55711.625000,5.571162e+04


El conjunto **indices** devuelve los índices de los puntos del conjunto de entrenamiento cuyas distancias quedan comprendidas en **distances**.

In [277]:
indices

,0,1,2
0,202651,202653,104721
1,202651,202653,104721
2,241943,251027,165909
3,210197,210196,137233
4,241943,251027,165909
...,...,...,...
694886,300760,300757,300754
694887,347426,330132,330130
694888,343493,343497,343495
694889,279045,279046,279047


# Sección 2: KNN mediante CPU

## Carga y tratamiento de datos

La lectura y tratamiento de los datos iniciales sigue el mismo proceso que para GPU:
1. Creamos un nuevo DataFrame vacío.
2. Recorremos los directorios disponibles.
3. Agregamos los CSV, convirtiendo columnas dispares a tipo *object* si fuera necesario.
4. Eliminamos duplicados.
5. Reordenamos el índice.

In [278]:
listings_cpu = pd.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df_cpu = pd.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df_cpu, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings_cpu.size == 0:
                    listings_cpu = temp_df_cpu
                else:
                    for column in listings_cpu.columns:
                        if listings_cpu[column].dtype != temp_df_cpu[column].dtype:
                            print('Found error: '+column+' type '+listings_cpu[column].dtype.name+' doesnt match '+temp_df_cpu[column].dtype.name)
                    listings_cpu = listings_cpu.append(temp_df_cpu)
                    
listings_cpu = listings_cpu.drop_duplicates().reset_index(drop=True)

Las funciones **type_conversion**, **column_factorize** y **clean_format_strings** son las mismas empleadas para GPU, ya que las operaciones subyacentes tienen la misma semántica y uso para cuDF/cupy como para pandas/numpy.

* **clean_format_price_cpu**: versión específica del tratado de cadenas de precios para CPU. Es necesario cambiar ligeramente el órden y número de operaciones para que numpy pueda ejecutar el mismo reemplazo de cadenas de precios que en GPU.

In [279]:
type_conversion(listings_cpu, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings_cpu, ['neighbourhood_cleansed'])

clean_format_strings(listings_cpu, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price_cpu(listings_cpu, ['price'])

La única diferencia en la conversión de coordenadas latitud-logitud a norte-este es que utilizamos funciones aritméticas provistas por numpy en lugar de cupy.

In [280]:
numpy_lat = listings_cpu['latitude'].to_numpy()
numpy_long = listings_cpu['longitude'].to_numpy()
n_numpy_array, e_numpy_array = latlong2osgbgrid_numpy(numpy_lat, numpy_long)
listings_cpu['northing'] = pd.Series(n_numpy_array).astype('float32')
listings_cpu['easting'] = pd.Series(e_numpy_array).astype('float32')

In [281]:
listings_cpu.head()

,host_id,host_response_rate,host_acceptance_rate,neighbourhood_cleansed,latitude,longitude,accommodates,price,number_of_reviews,reviews_per_month,northing,easting
0,3159.0,100.0,100.0,0.0,52.36575,4.94142,2.0,59.0,278.0,2.05,296936.50000,872297.875
1,32366.0,100.0,100.0,1.0,52.37802,4.89270,2.0,119.0,55.0,0.45,297980.56250,868856.625
2,59484.0,100.0,97.0,2.0,52.36509,4.89354,2.0,100.0,340.0,2.74,296550.62500,869051.125
3,56142.0,-1.0,53.0,1.0,52.37297,4.88339,3.0,125.0,5.0,0.18,297359.43750,868277.750
4,97647.0,100.0,100.0,1.0,52.38761,4.89188,2.0,155.0,217.0,2.14,299039.84375,868699.000


## Aplicación de KNN

La implementación de scikit-learn de *KNN* es idéntica en nuestro ejemplo.

In [282]:
%%time
knn_cpu = NearestNeighbors(n_neighbors=3)
listings_cheap_cpu = listings_cpu[listings_cpu['price'] <= 50.0].reset_index(drop=True)
listings_expensive_cpu = listings_cpu[listings_cpu['price'] >= 100.0].reset_index(drop=True)
listings_cheap_cols_cpu = listings_cheap_cpu[['easting', 'northing']]
listings_expensive_cols_cpu = listings_expensive_cpu[['easting', 'northing']]
knn_cpu.fit(listings_cheap_cols_cpu)
distances_cpu, indices_cpu = knn_cpu.kneighbors(listings_expensive_cols_cpu)

CPU times: user 1.08 s, sys: 4.42 ms, total: 1.08 s
Wall time: 1.08 s


In [283]:
distances_cpu

array([[ 59.87920784,  62.91652611,  62.91652611],
       [ 37.92782002,  37.92782002,  45.04481276],
       [ 72.93126714,  72.93126714,  72.93126714],
       ...,
       [ 92.29986035,  92.29986035, 113.55188604],
       [ 26.96929429,  39.15277771,  48.81371037],
       [ 98.57563844,  98.57563844,  98.57563844]])

In [284]:
indices_cpu

array([[  5361,   1382,   2949],
       [  4692,   5629,   1301],
       [  1571,   2023,   4843],
       ...,
       [334796, 326377, 336839],
       [329083, 329084, 337261],
       [331573, 334499, 342505]])

In [285]:
%reset -f